<a href="https://colab.research.google.com/github/myathetchai/bt4222-team3-jdsearch/blob/main/2_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing
In this file, we parsed the product metadata dataframe and queries dataframe to:
1. break the concatenated inputs into lists of individual inputs
2. convert the type of the inputs to the appropriate one(s).
3. further filter out the entries in the user behaviour dataframe that has would provide no meaningful inputs for model training

## Import Libraries

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
# import os
# from pathlib import Path
# import glob

## Load Sampled Dataset

In [ ]:
# mounting
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:

keep_cols = ["wid","name","brand_id","brand_name",
             "cate_id_1","cate_name_1","cate_id_2","cate_name_2",
             "cate_id_3","cate_name_3","cate_id_4","cate_name_4","shop_id"] # same as Sampling.ipynb, keep all columns

# Load filtered product metadata
product_df = pd.read_parquet(
    "/content/drive/MyDrive/BT4222 Group 3/1. Data Preparation/Data/product_filtered2",
    columns=keep_cols, dtype_backend="pyarrow"
)

print("Loaded product metadata:", product_df.shape)
product_df.head()

Loaded product metadata: (1263429, 13)


,wid,name,brand_id,brand_name,cate_id_1,cate_name_1,cate_id_2,cate_name_2,cate_id_3,cate_name_3,cate_id_4,cate_name_4,shop_id
0,90290292,89270446755918046025741917635585029871926...,75019956,8927044675591804,75739187,176355875210311,9682321,176355863043105,56674307,50298719,65420542,50298719,9644527
1,84294794,35314470278253305644862259056538602574191...,90358683,56448622,920973,413810773160222581402210,72011611,4138107765056023,77105805,2883009146686422,14729718,2883009146686422,51346695
2,43985934,99002012107475224750389807345717281391786...,99906610,99002012,24675518,2053319081620728,45178979,976474541190979,41488568,10747522807345713385795688352935,8059148,10747522807345713385795688352935,71737804
3,58357343,37269793566964962025051838249885733255404...,67566476,37269793417877055669649659865766,98432384,49629523385795642329532,78943759,42329532,84880674,25517595,16697032,25517595,45062963
4,71803457,53536632602574194190735260257419201104367...,78161242,53536632417877055360263159865766,6867541,50134995,47765755,290298641345793,66118678,41907352,70549743,41907352,24505854


In [ ]:
# check for null values for each column in the product dataframe
product_df.isnull().sum()

,0
wid,0
name,0
brand_id,0
brand_name,79063
cate_id_1,0
cate_name_1,0
cate_id_2,0
cate_name_2,0
cate_id_3,0
cate_name_3,0


Most features have no null values.

In [ ]:

# Load user behavior data
user_df = pd.read_csv(
    "/content/drive/MyDrive/BT4222 Group 3/1. Data Preparation/Data/user_behavior_data_sampled.csv", dtype=str)

print("Loaded user behavior data:", user_df.shape)
user_df.head()

Loaded user behavior data: (173831, 7)


,query,candidate_wid_list,candidate_label_list,history_qry_list,history_wid_list,history_type_list,history_time_list
0,63995226602574196100244625575117,36098226_82122191_34768767_71429327_87635543,1_0_0_0_0,-1_-1_-1_68784030_36547097602574193663289_95...,9227588_52788392_4585436_87422901_57364733_210...,ORD_ORD_CART_CART_CART_CART_CART_CART_CART_CAR...,0_0_22967659_82448_432_10315_51_188_51_254173_...
1,1254963427438884,30857980_44577431_82696473_94722703_51139903_8...,1_1_0_0_0_0_0_0_0_0,9755613572694219_46916741_-1_-1_-1_-1_-1_4927...,18256630_46206270_65230659_86878921_32743602_5...,ORD_ORD_ORD_CART_CART_CART_CART_ORD_CART_CART_...,0_4813478_1115463_14061983_920_467229_304_1197...
2,24374376067768939206305,42155506_90092899_66647496_92302381_6155258_87...,1_1_0_0_0_0_0_0_0_0,-1_-1,8133925_59478082,CLICK_CLICK,0_27_49660
3,29463948632095349356845164880503931105432...,97234676_83859812_90043344_51562568_19973201,1_0_0_0_0,3426234151020854598336931355834712920960_3...,49854616_56947170_16552251_91485122_25087051_7...,CLICK_CLICK_CLICK_CLICK_CLICK_CLICK_CLICK_CLIC...,0_12_4_6_4_14_6_6_4_611_58095_1524_481_20_363_...
4,2793914964365783663289,19711065_83045333_65555812_23738187_84489083_8...,1_1_0_0_0_0_0_0_0_0,25499566_24358647_606776896025741971768678_3...,68701_66196939_45800470_18419519_10090448_8073...,ORD_ORD_ORD_ORD_ORD_ORD_ORD_ORD_ORD_ORD_ORD_OR...,0_0_2015453_958482_0_79443_854291_13678_119386...


In [ ]:
# check for null values
user_df.isnull().sum()

,0
query,0
candidate_wid_list,2103
candidate_label_list,2103
history_qry_list,0
history_wid_list,0
history_type_list,0
history_time_list,0


Some query sessions has empty candidate lists due to our sampling strategy, which only retains products when there are positive interactions. Since we have a rich dataset with >173k query sessions, it is safe to remove this small subset during in model training process.

## Helper Functions

In [ ]:
def parse_candidate_list(candidate_str):
    """Parse candidate product IDs separated by underscores"""
    return candidate_str.split('_')

def parse_label_list(label_str):
    """Parse interaction labels as floats"""
    return [float(x) for x in label_str.split('_')]

def parse_history_queries(query_str):
    """Parse history query list, handling -1 for query-less interactions"""
    return query_str.split('_')

def parse_history_wids(wid_str):
    """Parse history product IDs"""
    return wid_str.split('_')

def parse_history_types(type_str):
    """Parse interaction types (ORD, CLICK, CART, etc.)"""
    return type_str.split('_')

def parse_time_list(time_str):
    """Parse time intervals as integers"""
    return [int(x) for x in time_str.split('_')]

## Parse User Behaviour Data
To enable feature engineering and model training, we split the concatenated variables (all except `query`) into lists.

In [ ]:
# Ensure the column is string type and handle potential NaN values
user_df['candidate_label_list'] = user_df['candidate_label_list'].astype(str).fillna('').apply(parse_label_list)
user_df['candidate_wid_list'] = user_df['candidate_wid_list'].astype(str).fillna('').apply(parse_candidate_list)
user_df['history_qry_list'] = user_df['history_qry_list'].astype(str).fillna('').apply(parse_history_queries)
user_df['history_wid_list'] = user_df['history_wid_list'].astype(str).fillna('').apply(parse_history_wids)
user_df['history_type_list'] = user_df['history_type_list'].astype(str).fillna('').apply(parse_history_types)
user_df['history_time_list'] = user_df['history_time_list'].astype(str).fillna('').apply(parse_time_list)

# check the parsed user_df
user_df.head()

,query,candidate_wid_list,candidate_label_list,history_qry_list,history_wid_list,history_type_list,history_time_list
0,63995226602574196100244625575117,"[36098226, 82122191, 34768767, 71429327, 87635...","[1.0, 0.0, 0.0, 0.0, 0.0]","[-1, -1, -1, 68784030, 365470976025741936632...","[9227588, 52788392, 4585436, 87422901, 5736473...","[ORD, ORD, CART, CART, CART, CART, CART, CART,...","[0, 0, 22967659, 82448, 432, 10315, 51, 188, 5..."
1,1254963427438884,"[30857980, 44577431, 82696473, 94722703, 51139...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9755613572694219, 46916741, -1, -1, -1, -1, ...","[18256630, 46206270, 65230659, 86878921, 32743...","[ORD, ORD, ORD, CART, CART, CART, CART, ORD, C...","[0, 4813478, 1115463, 14061983, 920, 467229, 3..."
2,24374376067768939206305,"[42155506, 90092899, 66647496, 92302381, 61552...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1, -1]","[8133925, 59478082]","[CLICK, CLICK]","[0, 27, 49660]"
3,29463948632095349356845164880503931105432...,"[97234676, 83859812, 90043344, 51562568, 19973...","[1.0, 0.0, 0.0, 0.0, 0.0]","[3426234151020854598336931355834712920960,...","[49854616, 56947170, 16552251, 91485122, 25087...","[CLICK, CLICK, CLICK, CLICK, CLICK, CLICK, CLI...","[0, 12, 4, 6, 4, 14, 6, 6, 4, 611, 58095, 1524..."
4,2793914964365783663289,"[19711065, 83045333, 65555812, 23738187, 84489...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[25499566, 24358647, 60677689602574197176867...","[68701, 66196939, 45800470, 18419519, 10090448...","[ORD, ORD, ORD, ORD, ORD, ORD, ORD, ORD, ORD, ...","[0, 0, 2015453, 958482, 0, 79443, 854291, 1367..."


## Filter for user history
Since the product metadata is sampled, we remove the products that do not exist in the product metadata from user interaction history, as there will be no meaningful information available for model trainig.

In [ ]:
# Calculate the number of interactions per user
history_interaction_count = user_df['history_wid_list'].apply(len)
print("Number of interactions per user:")
display(history_interaction_count.describe())

Number of interactions per user:


,history_wid_list
count,173831.000000
mean,153.409116
std,224.801556
min,1.000000
25%,29.000000
50%,78.000000
75%,188.000000
max,4802.000000


In [ ]:
# remove products that are not in the product metadata from the user's interaction history
product_wids = set(product_df['wid'])

def filter_history(row):
    filtered_history = [(wid, qry, typ, time) for wid, qry, typ, time in zip(row['history_wid_list'], row['history_qry_list'], row['history_type_list'], row['history_time_list']) if wid in product_wids]
    if filtered_history:
        wids, qrys, types, times = zip(*filtered_history)
        return pd.Series([list(wids), list(qrys), list(types), list(times)])
    else:
        return pd.Series([[], [], [], []])

user_df[['history_wid_list', 'history_qry_list', 'history_type_list', 'history_time_list']] = user_df.apply(filter_history, axis=1)

In [ ]:
user_df.head()

,query,candidate_wid_list,candidate_label_list,history_qry_list,history_wid_list,history_type_list,history_time_list
0,63995226602574196100244625575117,"[36098226, 82122191, 34768767, 71429327, 87635...","[1.0, 0.0, 0.0, 0.0, 0.0]","[-1, 68784030, 9592291398423633, 97447992, 97...","[9227588, 87422901, 21056095, 65178325, 549785...","[ORD, CART, CART, CART, CART, CART, CART, CART...","[0, 82448, 10315, 30, 33, 241, 18, 33, 3, 21, ..."
1,1254963427438884,"[30857980, 44577431, 82696473, 94722703, 51139...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9755613572694219, 46916741, -1, 26082679551...","[18256630, 46206270, 86878921, 82100823, 49359...","[ORD, ORD, CART, CART, CART, CART, ORD, ORD, C...","[0, 4813478, 14061983, 177223, 1304133, 42, 19..."
2,24374376067768939206305,"[42155506, 90092899, 66647496, 92302381, 61552...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1, -1]","[8133925, 59478082]","[CLICK, CLICK]","[0, 27]"
3,29463948632095349356845164880503931105432...,"[97234676, 83859812, 90043344, 51562568, 19973...","[1.0, 0.0, 0.0, 0.0, 0.0]","[3426234151020854598336931355834712920960,...","[49854616, 56947170, 16552251, 79406366, 73208...","[CLICK, CLICK, CLICK, CLICK, CLICK, CLICK, CLI...","[0, 12, 4, 14, 6, 58095, 1524, 481, 20, 363, 7..."
4,2793914964365783663289,"[19711065, 83045333, 65555812, 23738187, 84489...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[606776896025741971768678, 3989354160257419...","[45800470, 18419519, 30220497, 81499421, 52650...","[ORD, ORD, ORD, ORD, ORD, ORD, ORD, ORD, ORD, ...","[2015453, 958482, 13678, 1193864, 834215, 0, 0..."


In [ ]:
# checking the amount of empty user interaction history
empty_history_count = (user_df['history_wid_list'].apply(len) == 0).sum()
print(f"Number of users with empty history: {empty_history_count}")

# checking the new user interaction history per user
history_interaction_count = user_df['history_wid_list'].apply(len)
print("Number of interactions per user:")
display(history_interaction_count.describe())

Number of users with empty history: 4515
Number of interactions per user:


,history_wid_list
count,173831.000000
mean,65.699478
std,96.687828
min,0.000000
25%,11.000000
50%,32.000000
75%,81.000000
max,2396.000000


Additionally, we parse the candidate label as integer as another column. We also add a column which parse the history interaction as ordinal numbers: 1-CLICK, 1.5-FLW, 2-CART, 3-ORD. The numbers are chosen as such to align with the respective values for each type of interaction in the `candidate_label_list`

In [ ]:
user_df['candidate_label_list_int'] = user_df['candidate_label_list'].apply(lambda x: [int(label) for label in x if not pd.isna(label)])
user_df['history_type_list_ordinal'] = user_df['history_type_list'].apply(lambda x: [1 if label == 'CLICK' else 1.5 if label == 'FLW' else 2 if label == 'CART' else 3 for label in x if not pd.isna(label)])
user_df.head()

,query,candidate_wid_list,candidate_label_list,history_qry_list,history_wid_list,history_type_list,history_time_list,candidate_label_list_int,history_type_list_ordinal
0,63995226602574196100244625575117,"[36098226, 82122191, 34768767, 71429327, 87635...","[1.0, 0.0, 0.0, 0.0, 0.0]","[-1, 68784030, 9592291398423633, 97447992, 97...","[9227588, 87422901, 21056095, 65178325, 549785...","[ORD, CART, CART, CART, CART, CART, CART, CART...","[0, 82448, 10315, 30, 33, 241, 18, 33, 3, 21, ...","[1, 0, 0, 0, 0]","[3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,1254963427438884,"[30857980, 44577431, 82696473, 94722703, 51139...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9755613572694219, 46916741, -1, 26082679551...","[18256630, 46206270, 86878921, 82100823, 49359...","[ORD, ORD, CART, CART, CART, CART, ORD, ORD, C...","[0, 4813478, 14061983, 177223, 1304133, 42, 19...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[3, 3, 2, 2, 2, 2, 3, 3, 2, 2, 3, 2, 2, 1]"
2,24374376067768939206305,"[42155506, 90092899, 66647496, 92302381, 61552...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1, -1]","[8133925, 59478082]","[CLICK, CLICK]","[0, 27]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1]"
3,29463948632095349356845164880503931105432...,"[97234676, 83859812, 90043344, 51562568, 19973...","[1.0, 0.0, 0.0, 0.0, 0.0]","[3426234151020854598336931355834712920960,...","[49854616, 56947170, 16552251, 79406366, 73208...","[CLICK, CLICK, CLICK, CLICK, CLICK, CLICK, CLI...","[0, 12, 4, 14, 6, 58095, 1524, 481, 20, 363, 7...","[1, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,2793914964365783663289,"[19711065, 83045333, 65555812, 23738187, 84489...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[606776896025741971768678, 3989354160257419...","[45800470, 18419519, 30220497, 81499421, 52650...","[ORD, ORD, ORD, ORD, ORD, ORD, ORD, ORD, ORD, ...","[2015453, 958482, 13678, 1193864, 834215, 0, 0...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."


In [ ]:
# exporting the user_df as parquet file
user_df.to_parquet('/content/drive/MyDrive/BT4222 Group 3/1. Data Preparation/Data/user_behavior_data_sampled_parsed.parquet')
print("Exported user_df to parquet file")


Exported user_df to parquet file


## Parse Queries
We split the concatenated term_ids in `query` & `history_query_list` to a list of term_ids.

In [ ]:
# reading the middle parquet file
user_df = pd.read_parquet('/content/drive/MyDrive/BT4222 Group 3/1. Data Preparation/Data/user_behavior_data_sampled_parsed.parquet')
user_df.head()
print(user_df.shape)

(173831, 9)


In [ ]:
# parse query and each history query as a list of term_ids
user_df['query_list'] = user_df['query'].apply(lambda x: x.split('\x18'))
user_df['history_qry_list_terms'] = user_df['history_qry_list'].apply(lambda x: [query.split('\x18') for query in x])
user_df.head()

In [ ]:
# export as parquet file
user_df.to_parquet('/content/drive/MyDrive/BT4222 Group 3/1. Data Preparation/Data/user_behavior_data_sampled_parsed_2.parquet')
print("Exported user_df to parquet file")

## Parse product_metadata
Note: The session might crash here due to ram constraints. If this happens, re-run the cells to load the product_df again, and continue here.

We split the concatenated termids in the name columns to a list of term_ids


In [ ]:
# parse names as lists of term_ids
product_df['name_list'] = product_df['name'].astype(str).apply(lambda x: x.split('\x18'))
product_df['brand_name_list'] = product_df['brand_name'].astype(str).apply(lambda x: x.split('\x18'))
product_df['cate_name_1_list'] = product_df['cate_name_1'].astype(str).apply(lambda x: x.split('\x18'))
product_df['cate_name_2_list'] = product_df['cate_name_2'].astype(str).apply(lambda x: x.split('\x18'))
product_df['cate_name_3_list'] = product_df['cate_name_3'].astype(str).apply(lambda x: x.split('\x18'))
product_df['cate_name_4_list'] = product_df['cate_name_4'].astype(str).apply(lambda x: x.split('\x18'))

product_df.head()

In [ ]:
# export to a single parquet file
product_df.to_parquet('/content/drive/MyDrive/BT4222 Group 3/1. Data Preparation/Data/product_metadata_parsed.parquet')
print("Exported product_df to parquet file")

In [ ]:
product_df.info()